# Den klassiske bølgelikningen

Det viser seg at også klassiske bølger kan beskrives ved en differensiallikning:

$$
\nabla^2 u(\mathbf{x}, t) = -c^2 \frac{\partial^2}{\partial t^2} u(\mathbf{x}, t)
$$

hvor $\nabla^2$ er den såkalte Laplace operatoren, $u$ er bølgens høyde (amplitude), $\mathbf{x}$ er et romlig koordinat, $t$ er tiden og $c$ er en bølgekonstant.

Mange differensiallikninger er enkle å løse (tilnærmet) på datamaskinen om vi erstatter de deriverte med numeriske approksimasjoner. For å gjøre det litt spennende skal vi nå løse denne likningen direkte på bildet fra webcamet, og bruke mørke felter som "vegger". <a href="https://audunsh.github.io/projects/2015-11-19-wavecam/">Du kan lese en utfyllende forklaring av fremgangsmåten her.</a> Dette krever noen ekstra moduler i Python:

In [ ]:
!pip3 install opencv-python

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import bubblebox as bb


from matplotlib.animation import FuncAnimation

import numpy as np
from matplotlib.pyplot import *
import time
import cv2
from IPython.display import display, Image

class fwave():
    """
    Class to solve the classical wave equation:
    
    (d/dx)^2 u(x,t) = -c^2 (d/dt)^2 u(x,t)
    
    using a finite difference scheme.
    
    Author: Audun Skau Hansen (2015)
    
        x = discrete points to consider along the x axis 
        y = discrete points to consider along the y axis
        rho
        
    
    """
    def __init__(self, x,y,rho,b,q,I,V,f, dt):

        self.dt = dt
         
        self.dx = x[1]-x[0]
        self.x = x
        self.dy = y[1]-y[0]
        self.y = y
        self.rho = rho

        self.b = b
        self.beta =(1.0 + self.b*self.dt/(2.0*self.rho))**-1
        self.alpha = b*self.dt/(2.0*self.rho) - 1.0
        self.delta = (dt**2)/self.rho
        
        self.etax = (self.dt**2)/(2*self.rho*self.dx**2) 
        self.etay = (self.dt**2)/(2*self.rho*self.dy**2) 
        
        self.N = len(self.x)
        
        self.f = f
        self.u1 = I(self.x,self.y)
        self.RHS = np.zeros_like(self.u1) #ensure same shape
        self.V = V(self.x,self.y)
        self.q = q(self.x,self.y)
        self.dt = (self.q.max() *self.dx)
        self.t = 0
        
        
        
    def initialize(self, t0):
        #set up domain and conditions
        self.u1 = self.I(self.x,self.y)
        self.RHS = self.u1
        self.t = t0
        
    def advance(self):
        #advance solution one time step
        self.u = self.beta * (2*self.u1 + self.rhs() + self.alpha*self.u2)
        
        #impose boundary conditions
        self.impose_boundaries()
        
        #update system
        self.update()
                
    def first_step(self):
        #perform modified first step
        #self.u = self.beta * (self.eta*self.rhs() + self.delta*self.u1 - self.alpha*(self.u1 - 2*self.dt*self.V))
        #impose boundary conditions
        self.u =.5* self.beta * (2.0*self.u1 + self.rhs() + self.alpha*(2.0*self.dt*self.V))
        self.impose_boundaries()
        
        
        #update system
        self.update()
        
    def update(self):
        self.t += self.dt
        self.u2 = self.u1
        self.u1 = self.u
    
    #@autojit
    def rhs(self):


        q = self.q
        u1 = self.u1
        self.RHS *= 0

        self.RHS[1:-1,:] += (.5*(q[2:,:]+q[1:-1,:])*(u1[2:,:]-u1[1:-1,:]) - .5*(q[1:-1,:]+q[:-2,:])*(u1[1:-1,:]-u1[:-2,:]))*self.etax
        self.RHS[:,1:-1] += (.5*(q[:,2:]+q[:,1:-1])*(u1[:,2:]-u1[:,1:-1]) - .5*(q[:,1:-1]+q[:,:-2])*(u1[:,1:-1]-u1[:,:-2]))*self.etay

        return self.RHS + self.f(self.x, self.y, self.t)*self.delta        
        

    
    def solve(self, NT, display = False):
        self.first_step()
        for i in range(NT-1):
            self.advance()
            
            if display:
                cv2.imshow('frame',self.u)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                if cv2.waitKey(1) & 0xFF == ord('p'):
                    #Save picture to disk
                    a = time.strftime("%H:%M:%S")
                    print("Saved snapshot.")
                    cv2.imwrite("wavesim" +a+".png" , image)
            
    def impose_boundaries(self):
        self.u[:,0 ] = self.u[:,1 ]
        self.u[:,-1] = self.u[:,-2 ]
        self.u[0,: ] = self.u[1,: ]
        self.u[-1,:] = self.u[-2,: ]
    
    def f(self, t):
        return 0
    
class wavecam_lite():
    def __init__(self, video):
        
        self.cap = video
        #Ny, Nx = int(self.cap.get(3)/2), int(self.cap.get(4)/2)
        Ny, Nx = int(video.get(3)), int(video.get(4))

        #Nx, Ny = 480, 640
        Lx = Nx/10
        Ly = Ny/10
        x = np.linspace(0,Lx,Nx)
        y = np.linspace(0,Ly,Ny)

        I = lambda x,y: np.zeros((len(x), len(y))) #np.sin(x[:,None] + y[:,None].T)
        q = lambda x,y: 1 - .99*np.exp(-20*((x[:,None]-4)**2 + (y[:,None].T-3)**2)) 
        V = lambda x,y: np.zeros((len(x),len(y))) # a possible potential term
        f = lambda x,y,t: np.sin(1.8*t)*np.exp(-20*((x[:,None]-.2*Lx)**2 + (y[:,None].T-.5*Ly)**2)**2) 
        self.eq = fwave(x,y,1,0,q,I,V,f,0.1)
        self.eq.first_step()
        
        self.z = np.zeros_like(self.eq.u1)

    def advance(self, balance = .5):
        self.eq.advance()
        ret, frame = self.cap.read()
        self.im = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)/256.0
        self.eq.u1[self.im<.4] *= 0.5 #dirichlet boundary
        self.eq.u2[self.im<.4] *= 0.5 #dirichlet boundary
        
        
        self.z = balance*self.eq.u1 + (1-balance)*self.im
        return self.z
        
    

def run_video():
    video = cv2.VideoCapture(0)
    video.set(3, 100)
    video.set(4, 100)

    display_handle=display(None, display_id=True)

    wc = wavecam_lite(video)
    try:
        while True:
            #_, frame = video.read()
            #frame = cv2.flip(frame, 1) # if your camera reverses your image
            frame = wc.advance()*250

            _, frame = cv2.imencode('.jpeg', frame)
            display_handle.update(Image(data=frame.tobytes()))
    except KeyboardInterrupt:
        pass
    finally:
        video.release()
        display_handle.update(None)
        
run_video()

None

```{admonition} Diskutert gruppevis
- Hvordan utvikler bølgen seg i tid?
- Kan du identifisere noen av følgende bølgeegenskaper:
    - bølgelengde
    - frekvens
    - fase
    - hastighet
    - amplitude
- Hvordan varierer bølgeegenskapene som funksjon av tid?
- Kan du observere noen av følgende fenomener:
    - Refleksjon
    - Diffraksjon
    - Refraksjon
    - Interferens
- Rett kameraet mot tavla og tegn en boks til bølgene.
- Lag et hull i boksen og observer hvordan bølgene forlater boksen.
- Hva skjer om du lager to hull ved siden av hverandre?
```